#### <font color="grey">CE88 Fall 2017 - Section 2 (GSI: Sangjae Bae, sangjae.bae@berkeley.edu)</font>
<br>

# <center>**CE88 Lab Section 2**</center>
<center>September 18, 2017</center>

## Goals: an introduction to basic linear algebra tools in Python and analysis of Census data

## Agenda:
1. Basic Vector and Matrix Routines & Opertations
2. Shape Manipulation and Entry Selections
3. Minilab #2: Census Data
4. Quiz

### First some imports
Click in the box below and press 'Shift'+'Enter' to run the code.

In [ ]:
import numpy as np

## 1. Basic Vector and Matrix Routines & Operations

### Create "vectors" and "matrices"

In [ ]:
# Create Arrays
a = np.array([1,2,3])            # 1D array construction
b = np.array([[4,-5], [-2,3]])   # 2D array contruction
c = np.ones((5,4))               # 1D array of ones
d = np.zeros((5,4))              # 1D array of zeros
e = np.random.randn(4)           # 1D array of random normal entries

# Create Matrices
A = np.matrix(np.random.random((2,2)))            # 2x2 matrix of random values
B = np.asmatrix(b)                                # 2D array -> 2x2 matrix
C = np.mat(np.random.random((10,5)))              # 10x5 matrix of random values
D = np.mat([[3,4], [5,6]]);                       # 2x2 matrix from 2D list
E = np.identity(3);                               # 3x3 identity matrix
F = np.eye(3);                                    # 3x3 identity matrix
G = np.matrix(np.diag(np.random.rand(5)))         # 2D diagonal matrix (2D array)

# Check if array/matrix:
print (D.__class__)

### Vector and Matrix Operations

In [ ]:
# Create vectors, multi-dimensional arrays, and matrices
x = np.random.randn(4)
y = np.random.randn(5)
X = np.random.rand(5,4)
Y = np.random.rand(5,4)
M = np.random.randn(4,4)
A = np.matrix(np.random.randn(5,4))
B = np.matrix(np.random.randn(5,4))

# Array Operations -- ELEMENTWISE multiplication
X+Y, X-Y
W = X*Y
Z = X/Y    
X*x                         # multiply rows by x
np.dot(X,x)                 # Dot product
np.linalg.inv(M)
X.T, Y.T


# It is important to understand the difference between: X*x and A*x

# Matrix Operations
C = A+B          # C = np.add(A,B)
D = A-B          # D = np.subtract(A,B)
A.T, B.T
np.linalg.inv(A)

A*np.asmatrix(x).T          # Dot product; = np.dot(A,x) = np.inner(A,x)
np.multiply(A,x)            # Multiply rows by x

### Solving linear equations
Find x, y, z such that:<br><br>
x + 3y + 2z = 4 <br>
2x - y + z = 1 <br>
3x + y - 2z = 2

In [ ]:
A = np.array([[1,3,2], [2,-1,1],[3,1,-2]])
b = np.array([4,1,2])

np.linalg.inv(A)     # explicitly form inverse

# Q: x = np.linalg.inv(A) * b ?
# A: No, b is an array and hence elementwise multiplication.
#    The correct answers: 
#                         np.dot(np.linalg.inv(A), b) >> array
#                         np.linalg.inv(A) * np.asmatrix(b).T >> matrix


np.linalg.solve(A, b) # return A^(-1)*b, more efficient and numerically stable

## 2. Shape Manipulation and Entry Selections

### Shape Manipulation

In [ ]:
A = np.random.randn(4,4)
B = np.random.randn(2,4)
C = np.random.randn(4,2)
x = np.random.randn(5)
y = np.random.rand(5)

z = np.concatenate([x,y])  # concatenate 1D arrays

D = np.vstack([A,B])       # A and B stacked vertically >> must have same number of columns
E = np.hstack([A,C])       # A and C stacked horizontally >> must have same number of rows

# print(E)
# np.hsplit(D,2)             # Split the array horizontally at the 2nd index
# np.vsplit(E,2)             # Split the array vertically at the 2nd index

### Entry Selection

In [ ]:
A[0,0]   # select single entry
A[0,:]   # select entire column
A[0:3,1] # slice indexing

# integer indexing
idx_int = np.array([0,1,2])
A[idx_int,3]

print(A)
# boolean indexing
idx_bool = np.array([True, True, True, False]) # ==> 1,2,3 th rows

# fancy indexing on two dimensions
idx_bool2 = np.array([True, False, True, True]) # ==> 1,3,4 th columns
A[idx_bool, idx_bool2]     # not what you want
A[idx_bool,:][:,idx_bool2] # what you want

# Collecting columns with column names -- VERY IMPORTANT


## 3. Minilab #2: Census Data 
In the following lab we will take a look at some census data. A [census](https://en.wikipedia.org/wiki/Census) is the procedure of systematically acquiring and recording information about the members of a given population. The U.S. is required to take census data every 10 years. Information on the race, ethnicity, age, household size, family size etc. are recorded per [census tract](https://en.wikipedia.org/wiki/Census_tract).

In this lab we will look at the distribution of population age for a few Berkeley census tracts.

### Some imports

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline

### Next, reading the data

Sites like http://census.ire.org/ provide a nice interface to allow you to download census data. But we have downloaded the relevant data and cleaned it for you. Read in the csv below to see what the data looks like.

In [ ]:
data = Table.read_table('bay_area_census_age.csv')
 
data

### About the data
In the table above, we have total population, male population, female population, and population by age group for all the census tracts in the bay area. 

In addition we have some **geographic properties** of each census tract including the land area, water area, and latitude and longitude coordinates for a point inside the census tract.

Below is a map with the Bay Area census tracts highlighted in blue.

<img src="CA_census_tracts.jpg", width="500">

### Using a function

**Don't be scared by the code below** - we will get into the details later in the course. 
For now just recognize that the code below is a *function* used to compute the distance between two (latitude, longitude) coordinates. Rotate table is another function we will need later in the lab. Press 'Shift' + 'Enter' to run the code in the box below, we will use these functions shortly.

#### Computing the distance on a sphere aka great circle distance
For more detail please see https://en.wikipedia.org/wiki/Great-circle_distance


In [ ]:
def distance_on_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = np.pi/180.0
        
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
        
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
        
    # We can compute spherical distance from spherical coordinates.
    cos = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1-theta2)+
           np.cos(phi1)*np.cos(phi2))
    arc = np.arccos( cos )

    # Multiply arc by the radius of the earth to get length.
    return 3960.*arc #to get distance in miles

def rotate_table(table):
    '''transforms a 2 x n table to be an n x 2 table'''
    return Table().with_columns(['Columns', list(table.labels),
                                 'Values', list(table.to_array()[0])])

### Find the census tract closest to the Channing-Bowditch apartments (just South of Campus)
Now we will use the distance_on_sphere() function to find the census tract closest to the Channing-Bowditch apartments. From [Google Maps](https://goo.gl/maps/5xudrVbixun) we learn that the apartment is located at 37.867495, -122.257617 (lat, lon). We use the .apply() method to calculate the distance between each census tract and the Channing Bodwitch apartment.

In [ ]:
#return closest to 37.867495, -122.257617 (Channing-Bowditch apartments): https://goo.gl/maps/5xudrVbixun
lat1, lon1 = 37.867495, -122.257617

# calculate the distance from the Channing-Bowditch apartments to each tract. Save this in the data table 
# in a column labeled 'distance to Channing'
data['distance to Channing'] = data.apply(lambda lat2, lon2 : distance_on_sphere(lat1, lon1, lat2, lon2), 
                                          ['INTPTLAT10', 'INTPTLON10'])

#select the row where 'distance to Channing' is minimum. 
# This is the closest census tract to the Channing Apartments
channing_tract = data.where(data['distance to Channing'] == min(data['distance to Channing']))

#let's take a look at what this looks like.
channing_tract

### Create a horizontal bar graph of population vs. age group
We can use the barh function to create a bar graph. The function needs the data to be oriented in a single column. Right now the data is all oriented in one row. We will use the rotate table function (above) to rotate the table. We will save this table as a variable called tograph.

In [ ]:
tograph = channing_tract.select(['Under 5 years', '5 to 9 years', '10 to 14 years',
                                 '15 to 19 years','20 to 24 years','25 to 29 years',
                                 '30 to 34 years','35 to 39 years','40 to 44 years',
                                 '45 to 49 years','50 to 54 years','55 to 59 years',
                                 '60 to 64 years','65 to 69 years','70 to 74 years',
                                 '75 to 79 years','80 to 84 years','85 years and over'])
tograph = rotate_table(tograph)
tograph

### Run the code below to create the bar graph.

In [ ]:
tograph.relabel('Columns', 'Age group')
tograph.relabel('Values', 'Count')
tograph.barh('Age group')

**Question 1: ** What can we say about the data plotted above? Which age groups have the highest population. Do you think this is representative of the population for the rest of the Bay Area?

In [ ]:
# Answer here:

### Populations of merged age groups.
Now, we want to analyse the populations of the census track closest to the Channing-Bowditch apartment in terms of merged age groups. Specifically, we want to see how the population looks like for the age group: over 35 years. Print the table for the age group of over 35 years.



**Question 2: **What can you say about the populations of the age group?

In [ ]:
mergedGroup = channing_tract.select(['35 to 39 years','40 to 44 years',
                                 '45 to 49 years','50 to 54 years','55 to 59 years',
                                 '60 to 64 years','65 to 69 years','70 to 74 years',
                                 '75 to 79 years','80 to 84 years','85 years and over'])

# Solution #1
channing_tract['35 years and over'] = np.sum(np.asarray(mergedGroup.row(0)))

# Solution #2
popSum = np.zeros(len(mergedGroup.column(0)))

for col in mergedGroup.column_labels:
    popSum += mergedGroup.column(col)

channing_tract['35 years and over'] = popSum



channing_tract

### Populations of merged counties.
Now, we want to analyse the entire populations for the age group 20 to 24 over counties. Specifically, we want to see how the population looks like for the county group: county id <= 75. Print the table for the age group of county id <= 75.

** Question 3: ** What can you say about the populations of the county group?

In [ ]:
# Answer here:
data.where(data['COUNTY'] <= 75).column('20 to 24 years')
np.sum(data.where(data['COUNTY'] <= 75).column('20 to 24 years'))